# Cryptopals Challenges

## SET 01 BASIC

**Challenge 1** - Convert hex to base64

In [ ]:
#hex to b64 

import codecs

hex = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'

def hex2b64(hex):
  return codecs.encode(codecs.decode(hex, 'hex'), 'base64').decode()

print(f'base64: {hex2b64(hex)}')


base64: SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t



**Challenge 2** - Fixed XOR

In [ ]:
#xor 2 sequences of hex

def xor(first, second):
   return bytearray(x^y for x,y in zip(first, second))

HEX_1 = input("HEX 1: ")
HEX_2 = input("HEX 2: ")

D1 = bytearray.fromhex(HEX_1)
D2 = bytearray.fromhex(HEX_2)

r1 = xor(D1, D2)

print(r1.hex())




HEX 1: 6461
HEX 2: 6261
0600


**Challenge 3** - Single-byte XOR cipher


---


-	Try all the 256 possibilities of the XOR key (as the XOR key is a 8 bit length hex character, 28 = 256)

- Filter out the best result by a scoring function.

-	The scoring function grades a decoded string by the sum of its letter’s frequencies. 
- The higher the score is, the more it is likely to be an English message.


In [ ]:
import binascii



def scoring(str):
    # From https://en.wikipedia.org/wiki/Letter_frequency
    
    character_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000
    }
    score = 0
    for char in str.lower():
      if char in character_frequencies:
        score+=character_frequencies[char]
    return score


def single_byte_xor(encoded):
  plain = None
  key = None
  for xor_key in range(256):
    decoded = (''.join(chr(a ^ xor_key) for a in encoded))
    if plain == None or scoring(decoded) > scoring(plain):
      plain = decoded
      key = chr(xor_key)
  return plain, key


In [ ]:

HEX = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'

encoded = binascii.unhexlify(HEX)
print(single_byte_xor(encoded))

("Cooking MC's like a pound of bacon", 'X')


**Challenge 4** - Detect single-character XOR


---

- Slit the input string into substrings.
- Use a for loop to bruteforce each of the string, as challenge 3, and find the best string

- Split the input string into
substrings.

- Use a for loop to bruteforce each 
of the string, as challenge 3, and find the best string

In [ ]:
import binascii

def scoring(str):
    # From https://en.wikipedia.org/wiki/Letter_frequency
    character_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000
    }
    score = 0
    for char in str.lower():
      if char in character_frequencies:
        score+=character_frequencies[char]
    return score


def single_byte_xor(encoded,res):
  plain = res
  key = None
  for xor_key in range(256):
    decoded = (''.join(chr(a ^ xor_key) for a in encoded))
    if plain == None or scoring(decoded) > scoring(plain):
      plain = decoded
      key = chr(xor_key)
  return plain

In [ ]:
import requests
str = requests.get('https://cryptopals.com/static/challenge-data/4.txt').text
str = str.split('\n')
res =''

#perform a bruteforce for each of the sub-string
for i in str:
  encoded = binascii.unhexlify(i)
  res=single_byte_xor(encoded,res)

print(res)

Now that the party is jumping



**Challenge 5** - Implement repeating-key XOR

In [ ]:

def xor(first, second):
   return bytearray(x^y for x,y in zip(first, second))


HEX_1 = "Burning 'em, if you ain't quick and nimble I go crazy when I hear a cymbal"
print("Plaintext: ", HEX_1)
HEX_2 = input("Key: ")


def repeating_key_xor(D1, D2):  
  D1 = bytes(HEX_1, 'utf-8')
  D2 = bytes(HEX_2, 'utf-8')
  D2 = D2*(len(D1)//len(D2)+1) #repeat the key to the text size
  print (D2)
  return(xor(D1,D2))

print(repeating_key_xor(HEX_1,HEX_2).hex())

Plaintext:  Burning 'em, if you ain't quick and nimble I go crazy when I hear a cymbal
Key: ICE
b'ICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICEICE'
0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20690a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f


**Challenge 6** - Break repeating-key XOR


---


[Reference here.](https://laconicwolf.com/2018/06/30/cryptopals-challenge-6-break-repeating-key-xor/)

In [ ]:

import requests
import base64
#==================================================
def scoring(str):
    character_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000
    }
    score = 0
    for char in str.lower():
      if char in character_frequencies:
        score+=character_frequencies[char]
    return score

def single_byte_xor(encoded):
  key = None
  plain = None
  for xor_key in range(256):
    decoded = (''.join(chr(a ^ xor_key) for a in encoded))
    if plain == None or scoring(decoded) > scoring(plain):
      key =chr(xor_key)
      plain = decoded
  return key, plain
def xor(first, second):
   return bytearray(x^y for x,y in zip(first, second))

- Define Hamming distance calculating function.
> The Hamming distance is the amount of bits that differ. XOR two strings and count the number of bits 1. 

In [ ]:
#===================================================
#func to calculate hamming distance between 2 strings
def hammingdst(str1, str2):
  diff = 0
  for byte in xor(str1, str2):
    diff+= bin(byte).count("1")
  return diff



  
str1 = 'this is a test'
str2 = 'wokka wokka!!!'
str1 = bytes(str1, 'utf-8')
str2 = bytes(str2, 'utf-8')

print(hammingdst(str1, str2))

37


- Let KEYSIZE be the guessed length of the key.
 > try values from 2 to 40.




- Calculate Hamming distance of the ciphertext for each key size.
- Sort and store the results in a list

In [ ]:


#import cipher (in base64) and decode it into bytearray
cipher = requests.get('https://cryptopals.com/static/challenge-data/6.txt').text
cipher=bytearray(base64.b64decode(cipher.encode('ascii')).decode('ascii'), 'utf-8')

#the average distance value of each keysize case is stored in distances_list
distances_list = []

for test_size in range(2,40):
  text1 = cipher[:test_size]
  text2 = cipher[test_size: test_size*2]
  text3 = cipher[test_size*2: test_size*3]
  text4 = cipher[test_size*3: test_size*4]

  distance = (hammingdst(text1, text2) + hammingdst(text2, text3) + hammingdst(text3, text4))/(test_size*3)
  distances_list.append([test_size, distance])

print(distances_list)

distances_list = sorted(distances_list, key = lambda distances_list: distances_list[1])

[[2, 2.0], [3, 2.6666666666666665], [4, 3.4166666666666665], [5, 2.8], [6, 3.388888888888889], [7, 3.238095238095238], [8, 3.125], [9, 3.2222222222222223], [10, 3.466666666666667], [11, 3.242424242424242], [12, 3.4166666666666665], [13, 3.1025641025641026], [14, 3.1904761904761907], [15, 3.2222222222222223], [16, 3.3125], [17, 3.215686274509804], [18, 2.9444444444444446], [19, 3.1052631578947367], [20, 3.066666666666667], [21, 3.2857142857142856], [22, 3.4393939393939394], [23, 3.2753623188405796], [24, 3.1805555555555554], [25, 3.24], [26, 3.371794871794872], [27, 3.432098765432099], [28, 3.2261904761904763], [29, 2.793103448275862], [30, 3.188888888888889], [31, 3.032258064516129], [32, 3.40625], [33, 3.3434343434343434], [34, 3.2058823529411766], [35, 3.238095238095238], [36, 3.5277777777777777], [37, 3.2162162162162162], [38, 3.289473684210526], [39, 3.1452991452991452]]


- Select the 3 most likely keysize

In [ ]:

keysizes = (distances_list[0][0], distances_list[1][0], distances_list[2][0])


- Break the cipher into blocks of KEYSIZE length.
- Transpose them, make a block is the first byte of every blocks, a block is the second bytes of every blocks and so on.
- Treat each block as single-character XOR cipher in challenge 3. 
> Combine the single-character key of every block, now we got the repeating-key XOR key.


In [ ]:

#for each keysize case:

for KEYSIZE in keysizes:
  key=b''
  for i in range(KEYSIZE):
    #break into blocks and transpose them
    block = b''
    for j in range(i, len(cipher), KEYSIZE):
      block+=bytes([cipher[j]])
    
    key +=bytes(single_byte_xor(block)[0],('utf-8'))

  #repeat the key
    
  keyx = bytearray(key * (len(cipher)//len(key)+1))
  plaintext = bytes(xor(cipher, keyx))

  print(key)
  print(KEYSIZE)
  print(plaintext)


b'nn'
2
b's,q#ealq!} Rt\x07\x0bq\'r` }oei uox+Xnpo\'\nN:ss-]= \x0b<hn):rc+#ijb=|Rbpf\'tg\x7f!z"Ot)Enks)7\x7fjgn\nHi=nYxhbty/so<:^1nN}dk)!|&f+ \x0bPxgV+hkft(i!q7\x16\x10\x04\x0cXbsa/c&h;tuns,\x1ajpo\'tg\x7fl<\x14\x11\'n&Tnt}\'t!+&asc=jTo<woe/}hn"_1=\x0c{higi:ey/zx\'\x17][euoka(i!s \x16 &I<jib+6&f/n!N:f\x1aesw\'ln`x2n<^\x07\x0bq\'ll:noei l~=oH~{#lilq!u \x16^\x07X<dog:hig= l~=fU~hk\'aa~!UnT1)Er\'\n]!:l~=t!kx\x7f\x1abh#al`m-<"S nAe\'cf yc{:s!`r+0Fe#wo|id;=\x16 !\x0choe)=sbnnydkqbT,0#@o/L`r\'Z8/\x0c[h!)D\x10Uf!ouo=,Yjipb {r`hiEt:Dy\'wh7:O+9imk=i_+\x16Bid/sg<7Y!nHsi\'}n}o}+ `\'yjWe0#shjt!\x16\x19^-nUsr z:{tb \'!fi+Wn<\tTo/}dhnY2(\x0c;da|=\x7f&BncniiyUg<woe/iu})StDxtbrlii&e! ennxSe;#flcuvy*\x16^\x07\x0bq\'ignw\x7f+!wo\'mc[xy#\rTg\x7f!{\'D8\'Io\'shnc&\x7f&ex\'qdLn<nb nte<:^5:\x0cut f%:\x0cJ d!N=h[e<gfnl\x7f!~+B +^<shh :ge7 jny+T,<skav:\x0b\x16\x1dB5)I<5 $c:_n/ uox+Uey#~a(:v} X5n@uttl :rdn\nHs:x\x1adze\'mv:iy/Rt=C<ke}nnnnnbdfi+Jg}z\'tghni)^tD\x7fs\'I)-{h+(uol=bN+is\'aa~!q/]1nEh\'sf;tb+)onc=\x01

**Challenge 7** - AES in ECB mode


In [ ]:
#ECB mode

#!pip install pycryptodome
from Crypto.Cipher import AES
import requests

key =b'YELLOW SUBMARINE'
text = requests.get('https://cryptopals.com/static/challenge-data/7.txt').text
cipher = AES.new(key, AES.MODE_ECB)
print(cipher.decrypt(base64.b64decode(text)))


b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

**Challenge 8** - Detect AES in ECB mode

In [ ]:
#challenge 8 Detect AES in ECB mode
#ECB mode
import binascii
import requests

blocksize = 16

def has_rep_blocks(cipher, index, blocksize):
  cipher = binascii.unhexlify(cipher)
  #break the ciphertext into 16bytes blocks
  chunks = []
  for i in range(0,len(cipher), blocksize):
    chunks.append(cipher[i:blocksize + i])
  #total of block - distinct block + 1
  if len(chunks) - len(set(chunks)):
    print(f'Line #{index}:\n{cipher}')
    return True
  return False



In [ ]:
#get the ciphertext and devide it into a list of lines, seperate by '\n'
cipher = requests.get('https://cryptopals.com/static/challenge-data/8.txt').text
cipher = cipher.strip().split('\n')

#find the line which has replicated blocks
for line in cipher:
  has_rep_blocks(line, cipher.index(line), blocksize) 


Line #132:
b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j'


##SET 02 BLOCK CRYPTO

**Challenge 9** - Implement PKCS#7 padding

In [ ]:


def padding_pkcs7(buffer, block_size):
    padding = 0
    if len(buffer) % block_size:
        padding = int(len(buffer) / block_size + 1) * block_size - len(buffer)
    # Padding size must be less than a byte
    assert 0 <= padding <= 255
    return buffer + bytes([padding]*padding)

buffer = bytearray("YELLOW", 'utf-8')
print(padding_pkcs7(buffer, 10))


bytearray(b'YELLOW\x04\x04\x04\x04')


**Challenge 10** Implement CBC mode

In [ ]:
!pip install pycryptodome

     |████████████████████████████████| 1.9MB 2.9MB/s 


In [ ]:


#run the pip below in case cannot excute

#!pip install pycryptodome
import codecs
import binascii
import base64
import requests
from Crypto.Cipher import AES

def xor(first, second):
  return bytes(x^y for x,y in zip(first, second))

def AES_encrypt(key, content, mode):
  cryptor = AES.new( key,AES.MODE_ECB)
  last_block =b'\x00' * 16
  encrypted =b''
  for i in range(0, len(content), 16):
    last_block = cryptor.encrypt(xor(content[i : i + 16], last_block))
    encrypted += last_block
    if mode == 'ecb':
      last_block = b'\x00' * 16
  return encrypted

def AES_decrypt(key, content, mode):
  cryptor = AES.new(key, AES.MODE_ECB)
  last_block = b'\x00' * 16
  decrypted = b''
  for i in range(0, len(content), 16):
      decrypted += xor(cryptor.decrypt(content[i : i + 16]), last_block)
      if mode == 'cbc':
          last_block = content[i : i + 16]
  return decrypted


content = base64.b64decode(requests.get('https://cryptopals.com/static/challenge-data/10.txt').text)
de = AES_decrypt(b'YELLOW SUBMARINE', content, 'cbc')

print('plaintext here:\n', de)




plaintext here:
 b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can 

In [ ]:
#down here just to test if my code works

en = AES_encrypt(b'YELLOW SUBMARINE', de, 'cbc')
#en = binascii.hexlify(en)
#en = codecs.encode(codecs.decode(en, 'hex'), 'base64').decode()
de = AES_decrypt(b'YELLOW SUBMARINE', en, 'cbc')

print('\nrecover the plaintext, again:\n', de)


recover the plaintext, again:
 b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect an

**challenge 11** - An ECB/CBC detection oracle

In [ ]:


#run the pip below in case cannot excute

#!pip install pycryptodome
import codecs
import binascii
import base64
import os
from Crypto.Cipher import AES
import random
#====================================
# reused functions here
def padding_pkcs7(buffer, block_size):
    padding = 0
    if len(buffer) % block_size:
        padding = int(len(buffer) / block_size + 1) * block_size - len(buffer)
    # Padding size must be less than a byte
    assert 0 <= padding <= 255
    return buffer + bytes([padding]*padding)

def xor(first, second):
  return bytes(x^y for x,y in zip(first, second))

def AES_encrypt(key, content, mode):
  cryptor = AES.new( key, AES.MODE_ECB)
  if mode == 'ecb':
    last_block = b'\x00' * 16
  else:
    last_block = generatekey(16)
  encrypted =b''
  for i in range(0, len(content), 16):
    encrypted += cryptor.encrypt(xor(content[i : i + 16], last_block))
    if mode == 'cbc':
        last_block = content[i : i + 16]
  return encrypted

def AES_decrypt(key, content, mode):
  cryptor = AES.new(key, AES.MODE_ECB)
  if mode == 'ecb':
    last_block = b'\x00' * 16
  else:
    last_block = generatekey(16)
  decrypted = b''
  for i in range(0, len(content), 16):
      decrypted += xor(cryptor.decrypt(content[i : i + 16]), last_block)
      if mode == 'cbc':
          last_block = content[i : i + 16]
  return decrypted

#this func has been modified

def has_rep_blocks(cipher, _, blocksize):
  #cipher = binascii.unhexlify(cipher)
  #break the ciphertext into 16bytes blocks
  chunks = [cipher[i:blocksize + i] for i in range(0,len(cipher), blocksize)]
  #total of block - distinct block + 1
  if len(chunks) - len(set(chunks)) != 0:
    #print(sorted(chunks))
    return True
  return False

In [ ]:
#=================================




def generatekey(length):
  return os.urandom(length)

def encryption_oracle(content):
  if random.randint(0,1) == 1:
    mode = 'cbc'
  else: 
    mode ='ecb'
  content = generatekey(random.randint(5,10)) + content \
  + generatekey(random.randint(5,10))
  content = padding_pkcs7(content, 16)
  return AES_encrypt(generatekey(16), content, mode), mode
  


In [ ]:


for _ in range(40):
  message =b'a'*50
  message, mode = encryption_oracle(message)
  if has_rep_blocks(message, _, 16):
    if mode == 'ecb':
      print('detected: ECB, replicate blocks found')
    else:
      print('failed to detect')
  else:
    if mode == 'cbc':
      print('detected: CBC, no duplicate blocks found')
    else:
      print('failed to detect')

detected: ECB, replicate blocks found
detected: ECB, replicate blocks found
detected: ECB, replicate blocks found
detected: ECB, replicate blocks found
detected: CBC, no duplicate blocks found
detected: ECB, replicate blocks found
detected: CBC, no duplicate blocks found
detected: CBC, no duplicate blocks found
detected: CBC, no duplicate blocks found
detected: ECB, replicate blocks found
detected: CBC, no duplicate blocks found
detected: ECB, replicate blocks found
detected: CBC, no duplicate blocks found
detected: ECB, replicate blocks found
detected: ECB, replicate blocks found
detected: ECB, replicate blocks found
detected: ECB, replicate blocks found
detected: ECB, replicate blocks found
detected: ECB, replicate blocks found
detected: CBC, no duplicate blocks found
detected: CBC, no duplicate blocks found
detected: ECB, replicate blocks found
detected: CBC, no duplicate blocks found
detected: CBC, no duplicate blocks found
detected: CBC, no duplicate blocks found
detected: ECB, re

**Challenge 12** Byte at a time ECB decryption (Simple)

In [ ]:

#!pip install pycryptodome
import codecs
import binascii
import base64
import os
from Crypto.Cipher import AES
import random
#====================================
# reused functions section here
def padding_pkcs7(buffer, block_size):
    padding = 0
    if len(buffer) % block_size:
        padding = int(len(buffer) / block_size + 1) * block_size - len(buffer)
    # Padding size must be less than a byte
    assert 0 <= padding <= 255
    return buffer + bytes([padding]*padding)

def xor(first, second):
  return bytes(x^y for x,y in zip(first, second))

def AES_encrypt(key, content, mode):
  cryptor = AES.new( key,AES.MODE_ECB)
  last_block =b'\x00' * 16
  encrypted =b''
  for i in range(0, len(content), 16):
    last_block = cryptor.encrypt(xor(content[i : i + 16], last_block))
    encrypted += last_block
    if mode == 'ecb':
      last_block = b'\x00' * 16
  return encrypted

def AES_decrypt(key, content, mode):
  cryptor = AES.new(key, AES.MODE_ECB)
  last_block = b'\x00' * 16
  decrypted = b''
  for i in range(0, len(content), 16):
      decrypted += xor(cryptor.decrypt(content[i : i + 16]), last_block)
      if mode == 'cbc':
          last_block = content[i : i + 16]
  return decrypted

def generatekey(length):
  return os.urandom(length)

def encryption_oracle(content : bytes, key):
  content = content + unknown_string
  content = padding_pkcs7(content, 16)
  return AES_encrypt(key, content, 'ecb')





In [ ]:

def block_size_detector(key):
  for i in range(1, 40):
    sample = encryption_oracle(b'a'*i, key)
    if sample[:(i//2)] == sample[(i//2):i]:
      return i//2

def payload_size(message,blocksize):
  return len(padding_pkcs7(message, blocksize))//blocksize

def mode_detect(unknown_string, key, blocksize):
  sample = encryption_oracle(b'a'*(blocksize*2) + unknown_string , key)
  if sample[:blocksize] == sample[blocksize:blocksize*2]:
    return 'ecb'
  return 'failed to detect'

  


In [ ]:

global_key = generatekey(16)
unknown_string = base64.b64decode('Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg'
'aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq'
'dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg'
'YnkK'
)



block_size = block_size_detector(global_key)
print('block size:', block_size)
payload = payload_size(unknown_string, block_size)
print('payload:',payload)
mode = mode_detect(unknown_string, global_key, block_size)
print('mode:', mode)




block size: 16
payload: 9
mode: ecb


In [ ]:


def recover_one_more_byte_ecb(key , oracle, known_plaintext, block_size):
    
    # p+k+1 mod Bs = 0 hence p = (-k-1) mod Bs
    k = len(known_plaintext)
    padding_length = (-k-1) % block_size
    padding = b"A" * padding_length



    target_block_number = len(known_plaintext) // block_size
    target_slice = slice(target_block_number*block_size,
                         (target_block_number+1)*block_size)
    target_block = encryption_oracle(padding, key)[target_slice]



    # trying every possibility for the last byte
    for i in range(256):
        message = padding + known_plaintext + bytes([i])
        block = encryption_oracle(message, key)[target_slice]
        if block == target_block:
            return bytes([i])



def recover(key ,oracle, block_size):
    known_plaintext = b""

    payload_length = payload_size(oracle,16)*(block_size)

    for _ in range(payload_length):
        new_known_byte = recover_one_more_byte_ecb\
        (key, oracle, known_plaintext, block_size)
        known_plaintext = known_plaintext + new_known_byte
        
    return known_plaintext
  
  

In [ ]:
#================================


actual_unknown_string_len = len(unknown_string)
unknown_string = padding_pkcs7(unknown_string, 16)


encrypted = AES_encrypt(global_key, unknown_string, mode)
secret = recover(global_key , encrypted, block_size)[:actual_unknown_string_len]


print(secret.decode())


Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



**Challenge 13** ECB cut and paste

In [ ]:


#!pip install pycryptodome
import binascii
import base64
import codecs
import os
from Crypto.Cipher import AES
import random
#================================
#reuse functions section

def xor(first, second):
  return bytes(x^y for x,y in zip(first, second))

def AES_encrypt(key, content, mode):
  cryptor = AES.new( key,AES.MODE_ECB)
  last_block =b'\x00' * 16
  encrypted =b''
  for i in range(0, len(content), 16):
    last_block = cryptor.encrypt(xor(content[i : i + 16], last_block))
    encrypted += last_block
    if mode == 'ecb':
      last_block = b'\x00' * 16
  return encrypted

def AES_decrypt(key, content, mode):
  cryptor = AES.new(key, AES.MODE_ECB)
  last_block = b'\x00' * 16
  decrypted = b''
  for i in range(0, len(content), 16):
      decrypted += xor(cryptor.decrypt(content[i : i + 16]), last_block)
      if mode == 'cbc':
          last_block = content[i : i + 16]
  return decrypted

def generatekey(length):
  return os.urandom(length)

def padding_pkcs7(buffer, block_size):
    padding = 0
    if len(buffer) % block_size:
        padding = int(len(buffer) / block_size + 1) * block_size - len(buffer)
    # Padding size must be less than a byte
    assert 0 <= padding <= 255
    return buffer + bytes([padding]*padding)

def unpadding_pkcs7(s):
    if s[len(s)-s[-1]:] == s[-1:] * s[-1]: 
      return s[:-s[-1]]

In [ ]:

#================================

def str_to_dict(string):
  dict ={}
  for item in string.split("&"):
    item = item.split("=")
    dict[item[0]] = item[1]
  return dict

def dict_to_str(dict):
  return'&'.join(map('='.join, dict.items()))

def profile_for(email):
  return dict_to_str({
      'email': email.strip("&="),
      'uid': '10',
      'role': 'user'
  })



global_key = generatekey(16)
email = 'wutevahereisallgood@email.com'

encoded = bytearray(profile_for(email).encode())
print('given profile:\n',encoded)

encrypted = AES_encrypt(global_key, padding_pkcs7(encoded, 16), 'ecb')
encrypted_ = encrypted[:16*2]

seed = AES_encrypt(global_key, padding_pkcs7(b'admin', 16), 'ecb')
encrypted_ = encrypted_ + seed
print('poisoned profile:\n',bytearray(unpadding_pkcs7(AES_decrypt(global_key, encrypted_, 'ecb'))))

given profile:
 bytearray(b'email=bad@email.com&uid=10&role=user')
poisoned profile:
 bytearray(b'email=bad@email.com&uid=10&role=admin')


**Challenge 15** - PKCS#7 padding validation

In [ ]:


def unpadding_pkcs7_validation(s: bytes) -> bytes:
    if s[len(s)-s[-1]:] != s[-1:] * s[-1]: 
      raise Exception("Bad padding occurred")
    return s[:-s[-1]]

s = b"ICE ICE BABY\x04\x04\x04\x04"
print(unpadding_pkcs7_validation(s))

b'ICE ICE BABY'


In [ ]:


#throw exception:

s = b"ICE ICE BABY\x04\x04\x04\x05"
print(unpadding_pkcs7_validation(s))

Exception: ignored

**Challenge 16** - CBC bitflipping attacks

In [ ]:


#!pip install pycryptodome
import binascii
import base64
import codecs
import os
from Crypto.Cipher import AES
import random


#========================================
def xor(first, second):
  return bytes(x^y for x,y in zip(first, second))

def AES_encrypt(key, content, mode):
  cryptor = AES.new( key,AES.MODE_ECB)
  last_block =b'\x00' * 16
  encrypted =b''
  for i in range(0, len(content), 16):
    last_block = cryptor.encrypt(xor(content[i : i + 16], last_block))
    encrypted += last_block
    if mode == 'ecb':
      last_block = b'\x00' * 16
  return encrypted


def AES_decrypt(key, content, mode):
  cryptor = AES.new(key, AES.MODE_ECB)
  last_block = b'\x00' * 16
  decrypted = b''
  for i in range(0, len(content), 16):
      decrypted += xor(cryptor.decrypt(content[i : i + 16]), last_block)
      if mode == 'cbc':
          last_block = content[i : i + 16]
  return decrypted


def padding_pkcs7(buffer, block_size):
  padding = 0
  if len(buffer) % block_size:
      padding = int(len(buffer) / block_size + 1) * block_size - len(buffer)
    # Padding size must be less than a byte
  assert 0 <= padding <= 255
  return buffer + bytes([padding]*padding)


def unpadding_pkcs7(s):
    if s[len(s)-s[-1]:] == s[-1:] * s[-1]: 
      return s[:-s[-1]]


def generate_key():
  return os.urandom(16)
#=================================

In [ ]:


def encrypt_(s) -> bytes:
    s = prefix + s.replace(b'=', b'').replace(b';', b'') + suffix
    s = padding_pkcs7(s, 16)
    return AES_encrypt(random_key, s, 'cbc')


def is_admin(s: bytes) -> bool:
    return b'admin=true' in unpadding_pkcs7(AES_decrypt(random_key, s, 'cbc'))



def bitflip(s):
  cipher = encrypt_(s)
  bitflip = xor(cipher[16:32],
    xor(s, b"uwuu;admin=true;"))
  return cipher[:16] + bitflip + cipher[32:]

#~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:




random_key = generate_key()

prefix = b"comment1=cooking%20MCs;userdata="
suffix = b";comment2=%20like%20a%20pound%20of%20bacon"


s = b'uwuu;admin=true;'

before = encrypt_(s)
print(unpadding_pkcs7(AES_decrypt(random_key, before,'cbc')))
print(is_admin(before))


b'comment1=cooking%20MCs;userdata=uwuuadmintrue;comment2=%20like%20a%20pound%20of%20bacon'
False


In [ ]:



s = b'uwuuAadminAtrueA'
after = bitflip(s)
print(unpadding_pkcs7(AES_decrypt(random_key, after,'cbc')))
print(is_admin(after))



b'comment1=cooking\x1b\x17\xb4\xa1\x1fgJ\xc8\xf5,\x0b\xca\xa8-U\xf7uwuu;admin=true;;comment2=%20like%20a%20pound%20of%20bacon'
True


In [ ]:

random_key = generate_key()

prefix = b"comment1=cooking%20MCs;userdata="
suffix = b";comment2=%20like%20a%20pound%20of%20bacon"


def bitflip(s):
  cipher = encrypt_(s)
  bitflip = xor(cipher[16:32],
    xor(s, b"uwuu;admin=true;"))
  return cipher[:16] + bitflip + cipher[32:]


s = b'uwuuAadminAtrueA'
bf = bitflip(s)
print(unpadding_pkcs7(AES_decrypt(random_key, bf,'cbc'))[16:])
print(is_admin(bf))

b'i\xeev\x87\x8c\x8d|\xbc{\x91Du\x1e\xfa\x1f\xb5uwuu;admin=true;;comment2=%20like%20a%20pound%20of%20bacon'
True


##SET 03

**Challenge 18** - implement CTR, the stream cipher mode

In [ ]:


#!pip install pycryptodome
import binascii
import base64
import codecs
import os
from Crypto.Cipher import AES
import random

#==========================================

def xor(first, second):
  return bytes(x^y for x,y in zip(first, second))

def AES_encrypt(key, content, mode):
  cryptor = AES.new( key,AES.MODE_ECB)
  last_block =b'\x00' * 16
  encrypted =b''
  for i in range(0, len(content), 16):
    last_block = cryptor.encrypt(xor(content[i : i + 16], last_block))
    encrypted += last_block
    if mode == 'ecb':
      last_block = b'\x00' * 16
  return encrypted

def AES_decrypt(key, content, mode):
  cryptor = AES.new(key, AES.MODE_ECB)
  last_block = b'\x00' * 16
  decrypted = b''
  for i in range(0, len(content), 16):
      decrypted += xor(cryptor.decrypt(content[i : i + 16]), last_block)
      if mode == 'cbc':
          last_block = content[i : i + 16]
  return decrypted

#===============================================


def encrypt_CTR(key, cipher, iv = b'\x00'*8 , nonce = 0):
  ctr = b''
  for i in range(0, len(cipher), 16):
    block = cipher[i: i + 16]
    current_iv = (nonce.to_bytes(8, byteorder ='big') + iv)[::-1] 
   
    ctr += xor(block, AES_encrypt(key, current_iv, 'ecb'))
    nonce+=1
  return ctr

#=================================================

encrypt_CTR(b"YELLOW SUBMARINE",
base64.b64decode('L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ=='))


b"Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby "

**Challenge 26** - CTR bitflipping

In [ ]:


#all i did was edit that challenge 16 so dont ask me anything :(

#!pip install pycryptodome
import binascii
import base64
import codecs
import os
from Crypto.Cipher import AES
import random

#========================================
def xor(first, second):
  return bytes(x^y for x,y in zip(first, second))

def AES_encrypt(key, content, mode):
  cryptor = AES.new( key,AES.MODE_ECB)
  last_block =b'\x00' * 16
  encrypted =b''
  for i in range(0, len(content), 16):
    last_block = cryptor.encrypt(xor(content[i : i + 16], last_block))
    encrypted += last_block
    if mode == 'ecb':
      last_block = b'\x00' * 16
  return encrypted

def AES_decrypt(key, content, mode):
  cryptor = AES.new(key, AES.MODE_ECB)
  last_block = b'\x00' * 16
  decrypted = b''
  for i in range(0, len(content), 16):
      decrypted += xor(cryptor.decrypt(content[i : i + 16]), last_block)
      if mode == 'cbc':
          last_block = content[i : i + 16]
  return decrypted


def padding_pkcs7(buffer, block_size):
  padding = 0
  if len(buffer) % block_size:
      padding = (int(len(buffer) / block_size + 1) * block_size) - len(buffer)
    # Padding size must be less than a byte
  assert 0 <= padding <= 255
  return buffer + bytes([padding]*padding)

def unpadding_pkcs7(s):
    if s[len(s)-s[-1]:] == s[-1:] * s[-1]: 
      return s[:-s[-1]]

def generate_key():
  return os.urandom(16)
#=================================



In [ ]:



def encrypt_(s) -> bytes:
  s = prefix + s.replace(b'=', b'').replace(b';', b'') + suffix
  return encrypt_CTR(random_key, s)

def is_admin(s: bytes) -> bool:
    return b'admin=true' in encrypt_CTR(random_key, s)

def bitflip(s):
  cipher = encrypt_(s)
  bitflip = xor(cipher[32:48], #the admin block itself
    xor(s, b"uwuu;admin=true;"))
  return cipher[:32] + bitflip + cipher[48:]

def encrypt_CTR(key, cipher, iv = b'\x00'*8 , nonce = 0):
  rtrn = b''
  for i in range(0, len(cipher), 16):
    block = cipher[i: i + 16]
    current_iv = (nonce.to_bytes(8, byteorder ='big') + iv)[::-1]
    rtrn += xor(block, AES_encrypt(key, current_iv, 'ecb'))
    nonce+=1
  return rtrn
#~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:

random_key = generate_key()

prefix = b"comment1=cooking%20MCs;userdata="
suffix = b";comment2=%20like%20a%20pound%20of%20bacon"


s = b'uwuu;admin=true;'

before = encrypt_(s)
print(encrypt_CTR(random_key,before))
print(is_admin(before))


b'comment1=cooking%20MCs;userdata=uwuuadmintrue;comment2=%20like%20a%20pound%20of%20bacon'
False


In [ ]:



s = b'uwuuAadminAtrueA'
after = bitflip(s)
print(encrypt_CTR(random_key,after))
print(is_admin(after))


b'comment1=cooking%20MCs;userdata=uwuu;admin=true;;comment2=%20like%20a%20pound%20of%20bacon'
True


**Challenge 27** - Recover the key from CBC with IV=Key

In [ ]:

#!pip install pycryptodome
import binascii
import base64
import codecs
import os
from Crypto.Cipher import AES
import random




#========================================
def xor(first, second):
  return bytes(x^y for x,y in zip(first, second))

def AES_encrypt(key, content, mode):
  cryptor = AES.new( key,AES.MODE_ECB)
  last_block = key
  encrypted =b''
  for i in range(0, len(content), 16):
    last_block = cryptor.encrypt(xor(content[i : i + 16], last_block))
    encrypted += last_block
  return encrypted


def AES_decrypt(key, content, mode):
  cryptor = AES.new(key, AES.MODE_ECB)
  last_block = key
  decrypted = b''
  for i in range(0, len(content), 16):
      decrypted += xor(cryptor.decrypt(content[i : i + 16]), last_block)
      if mode == 'cbc':
          last_block = content[i : i + 16]
  return decrypted


def padding_pkcs7(buffer, block_size):
  padding = 0
  if len(buffer) % block_size:
      padding = int(len(buffer) / block_size + 1) * block_size - len(buffer)
    # Padding size must be less than a byte
  assert 0 <= padding <= 255
  return buffer + bytes([padding]*padding)



def unpadding_pkcs7(s):
    if s[len(s)-s[-1]:] == s[-1:] * s[-1]: 
      return s[:-s[-1]]

def generate_key():
  return os.urandom(16)

#===================================

In [ ]:




random_key = generate_key()
msg = b'A'*16 + b'B'*16 + b'C'*16
print(msg)



encrypted = AES_encrypt(random_key, msg, 'cbc')




encrypted = encrypted[:16] + b'\x00'*16 + encrypted [:16]


attacking = AES_decrypt(random_key, encrypted, 'cbc')

recover = xor(attacking[0:16], attacking[32:48])



if AES_encrypt(recover, msg, 'cbc') ==  AES_encrypt(random_key, msg, 'cbc'):
  print('key recovered successfully:\n', recover)
else:
  print('failed to recover')

b'AAAAAAAAAAAAAAAABBBBBBBBBBBBBBBBCCCCCCCCCCCCCCCC'
key recovered successfully:
 b'\x907\x8e\x91\x00\xad\xc1h7\xeb\xd1\xa7\xb1\x88\xd8\xa3'


**Challenge 33** - Implement Diffie-Hellman

In [ ]:
import random

p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
q = p // 2 - 1

a = random.randint(0, q - 1)
A = pow(g,a,p)
b = random.randint(0, q - 1)
B = pow(g,b,p)

print(f"a = {a}")
print(f"b = {b}")
print(f"A = {A}")
print(f"B = {B}")

sB = pow(B,a,p)
sA = pow(A,b,p)

if sA != sB:
        print("Error occurred!")
print(f"sA = {sA}")

a = 581318229531928184037987825166046568411107488613067520851552155733023152553498422661906520990077592104261546871638335483710272808559899502712916586208643002821101890488485164727553345530654986096751530022648332243149029416659676418523247884318912337760912714201284528635459846629167022352651826626006615126475460955950861105867370651385690231705345760431148116020537360594674405455812869061192142984500554397797334279049395043403828122845630404792303505680952959
b = 1103684322651188555880570119260739174646714055629454730252322012923827364225789713161110602001159677074630339885434552451470732896336788914453482515429644046630511125750791431426112513047260313564678724248014087399013878056533717623452389442419340161573424164649505588907588706330753108532121043890820664797451252441456771173805408008873266952733750974102839006077433778150719732991495154514805145961433181690296989500071687718545148205538284909776317769797997065
A = 3708617992277913010747725192751129693155602685774963366255720

Additional requirement of this task is to build ourselves power function:

In [ ]:
def power(x, y, p):
  res = 1
  # Update x if it is more than or equal to p
  x = x % p

  while (y > 0):
    # If y is odd, multiply x with result
    if (y & 1):
      res = (res * x) % p
      # y must be even now
      y = y>>1; # y = y/2
      x = (x * x) % p
  return res

if power(5, 3, 37) == pow(5, 3, 37):
  print('qualified')

qualified


**Challenge 39** - Implement RSA

Firstly, implement 2 big prime numbers, qualified by miller-rabin algorithm:

In [ ]:
import random

def millerTest(d, n):
  a = 2 + random.randint(1, n - 4);
  x = pow(a, d, n)
  if (x == 1 or x == n - 1):
    return True;
  while (d != n - 1):
    x = (x * x) % n
    d *= 2;
    if (x == 1):
      return False
    if (x == n - 1):
      return True
  return False


def isPrime( n, k = 50):
  if (n <= 1 or n == 4):
    return False
  if (n <= 3):
    return True
  d = n - 1;
  while (d % 2 == 0):
    d //= 2
  for i in range(k):
    if (millerTest(d, n) == False):
      return False
  return True


def generate_odd_int(len):
  return ((random.getrandbits(len*8))|(1<<len -1))|1

def generate_prime(len):
  p = generate_odd_int(len)
  while not isPrime(p, k):
    p = generate_odd_int(len)
  return p



modulo-inverse to compute d:

In [ ]:
def invmod(e, _phi):
  d = -1
  y0, y1, phi = 0, 1, _phi
  while (e > 0):
    r = phi % e
    if r == 0:
      break
    q = phi//e
    d = y0 - y1*q

    phi, e, y0, y1 = e, r, y1, d
  while d < 0:
    d = d + _phi
  return d

#e.g:

print(invmod(65537,4145748157138759360))

3865388118802567873


In [ ]:
def RSA_cipher(msg, PU):
  return pow(msg,PU[0], PU[1])

In [ ]:
k = 50

p = generate_prime(256)
q = generate_prime(256)
while p == q: q = generate_prime(256)
#e = generate_relative_prime(p,q)

n = p*q
phi = (p-1)*(q-1)

e = 3
d = invmod(3, phi)
PU = (e, n)
PR = (d, n)

In [ ]:

message = "The University of Information Technology"
print(f'original message:  {message}')
message = int(message.encode('utf-8').hex(), 16)

en = RSA_cipher(message, PU)
de = RSA_cipher(en,PR)

de = hex(de)[2:]
print(f'encrypted message: {en}')
print(f'decrypted message: {de}')
print(f'recovered message: {bytearray.fromhex(de).decode()}')


original message:  The University of Information Technology
encrypted message: 349320101287431902690575625957890278004172188544879664065955925932979766873730870723193735721924684335223291421859359152996656405528831815483687170454911750815191268403758166870698430990219525822450024839679750898489453478074097108276310488681309588320208230805220191268017674316355939625
decrypted message: 54686520556e6976657273697479206f6620496e666f726d6174696f6e20546563686e6f6c6f6779
recovered message: The University of Information Technology


**Challenge 40 - Implement an E=3 RSA Broadcast attack**

In [ ]:
def genKey(size):
  k = 50
  p = generate_prime(size)
  q = generate_prime(size)

  while p == q: q = generate_prime(size)
  
  n = p*q
  phi = (p-1)*(q-1)
  e = 3
  d = modinv(3, phi)
  PU = (e, n)
  PR = (d, n)
  return PU, PR

In [ ]:
(pub0, priv0) = genKey(128)
(pub1, priv1) = genKey(128)
(pub2, priv2) = genKey(128)

In [ ]:

import math
plaintext = 'cryptography is very difficult'
plainnum = int(plaintext.encode('utf-8').hex(), 16)

c0 = RSA_cipher(plainnum, pub0)
c1 = RSA_cipher(plainnum, pub1)
c2 = RSA_cipher(plainnum, pub2)

n0 = pub0[1]
n1 = pub1[1]
n2 = pub2[1]

ms0 = n1 * n2
ms1 = n0 * n2
ms2 = n0 * n1

N = n0 * n1 * n2

r0 = (c0 * ms0 * invmod(ms0, n0))
r1 = (c1 * ms1 * invmod(ms1, n1))
r2 = (c2 * ms2 * invmod(ms2, n2))

r = (r0 + r1 + r2) % N

def floor_Root(n, s):
    b = n.bit_length()
    p = math.ceil(b/s)
    x = 2**p
    while x > 1:
        y = (((s - 1) * x) + (n // (x**(s-1)))) // s
        if y >= x:
            return x
        x = y
    return 1

m = floor_Root(r, 3)
mstr = bytearray.fromhex(hex(m)[2:]).decode()

if mstr != plaintext:
  raise Exception(mstr + b' != ' + plaintext)

print(mstr)

cryptography is very difficult


#Rootmee Challenges


ROOTme

**Encoding ASCII**

In [ ]:

import binascii

v = b'495f6c3076335f4372797074305f3a29'

v = binascii.unhexlify(v)

print(v)

b'I_l0v3_Crypt0_:)'


**PIXELS MADNESS**

In [ ]:
import requests

result=""
#lines = requests.get('http://challenge01.root-me.org/cryptanalyse/ch4').text

lines ="0x3+1x1+0x1+0x1+0x7+1x2+0x15+1x1+0x8+1x1+0x8+1x1+0x1+1x1+0x1+1x1+0x1+1x1+0x1+1x1+0x3+1x1+0x1+1x1+0x3+1x1+0x1+1x4+0x2+1x1+0x25\n0x2+1x1+0x4+1x1+0x4+1x3+0x1+1x2+0x2+1x8+0x11+1x4+0x1+1x3+0x6+1x2+0x4+1x1+0x4+1x2+0x7+1x4+0x4+1x2+0x7+1x2+0x3+1x2+0x3\n0x3+1x1+0x2+1x1+0x2+1x1+0x11+1x2+0x2+1x3+0x7+1x1+0x4+1x2+0x2+1x2+0x7+1x1+0x6+1x1+0x2+1x1+0x4+1x3+0x1+1x1+0x4+1x1+0x2+1x1+0x2+1x1+0x3+1x1+0x2+1x3+0x2+1x2+0x3\n1x1+0x2+1x1+0x4+1x1+0x2+1x1+0x1+1x1+0x2+1x1+0x2+1x1+0x1+1x2+0x2+1x2+0x1+1x2+0x3+1x1+0x3+1x1+0x2+1x2+0x1+1x3+0x3+1x1+0x2+1x1+0x4+1x2+0x1+1x1+0x4+1x1+0x3+1x2+0x12+1x2+0x1+1x1+0x3+1x7+0x3\n0x3+1x1+0x7+1x1+0x1+1x1+0x4+1x1+0x2+1x2+0x2+1x2+0x4+1x1+0x2+1x1+0x1+1x2+0x1+1x8+0x1+1x1+0x4+1x1+0x5+1x1+0x3+1x2+0x2+1x1+0x1+1x2+0x2+1x1+0x3+1x2+0x9+1x1+0x1+1x2+0x2+1x3+0x2+1x1\n0x7+1x1+0x4+1x1+0x4+1x1+0x1+1x1+0x1+1x7+0x3+1x1+0x1+1x2+0x3+1x1+0x1+1x6+0x1+1x1+0x3+1x1+0x2+1x1+0x14+1x2+0x8+1x1+0x10+1x2+0x3+1x2+0x1+1x1+0x1\n0x6+1x5+0x4+1x1+0x7+1x1+0x2+1x1+0x3+1x2+0x4+1x1+0x8+1x1+0x3+1x2+0x1+1x2+0x3+1x1+0x8+1x1+0x2+1x2+0x1+1x1+0x3+1x7+0x5+1x2+0x2+1x1+0x2+1x2+0x3\n0x1+1x1+0x2+1x1+0x1+1x2+0x5+1x1+0x6+1x2+0x3+1x1+0x2+1x1+0x1+1x2+0x20+1x8+0x1+1x1+0x1+1x1+0x4+1x2+0x3+1x1+0x2+1x2+0x3+1x2+0x7+1x2+0x3+1x2+0x4\n0x2+1x1+0x3+1x5+0x5+1x2+0x7+1x1+0x4+1x2+0x2+1x1+0x2+1x2+0x1+1x1+0x3+1x1+0x6+1x2+0x2+1x2+0x3+1x2+0x2+1x3+0x1+1x1+0x6+1x3+0x3+1x5+0x3+1x1+0x4+1x1+0x5\n0x4+1x2+0x3+1x2+0x3+1x1+0x5+1x2+0x2+1x1+0x1+1x1+0x1+1x1+0x1+1x2+0x9+1x1+0x3+1x1+0x2+1x1+0x1+1x1+0x2+1x1+0x1+1x2+0x2+1x1+0x2+1x1+0x1+1x1+0x4+1x3+0x1+1x1+0x2+1x2+0x3+1x2+0x3+1x1+0x5+1x1+0x4+1x1+0x2\n0x6+1x5+0x4+1x1+0x1+1x1+0x2+1x2+0x6+1x1+0x1+1x7+0x4+1x3+0x3+1x1+0x4+1x1+0x2+1x2+0x4+1x1+0x6+1x1+0x6+1x8+0x3+1x1+0x5+1x1+0x7\n0x2+1x1+0x3+1x6+0x4+1x1+0x1+1x3+0x4+1x1+0x2+1x2+0x4+1x1+0x5+1x1+0x2+1x1+0x3+1x2+0x3+1x1+0x2+1x3+0x1+1x1+0x2+1x2+0x3+1x3+0x2+1x3+0x9+1x1+0x4+1x2+0x7+1x2"
lists = lines.split("\n")

for i in range(len(lists)):
  lists[i] = lists[i].split("+")
  for j in range(len(lists[i])):
    result+= lists[i][j][0] * int(lists[i][j][2:])
  result+="\n"
result = result.replace("0"," ")
result = result.replace("1","█")
print(result)



   █         ██               █        █        █ █ █ █ █   █ █   █ ████  █                         
  █    █    ███ ██  ████████           ████ ███      ██    █    ██       ████    ██       ██   ██   
   █  █  █           ██  ███       █    ██  ██       █      █  █    ███ █    █  █  █   █  ███  ██   
█  █    █  █ █  █  █ ██  ██ ██   █   █  ██ ███   █  █    ██ █    █   ██            ██ █   ███████   
   █       █ █    █  ██  ██    █  █ ██ ████████ █    █     █   ██  █ ██  █   ██         █ ██  ███  █
       █    █    █ █ ███████   █ ██   █ ██████ █   █  █              ██        █          ██   ██ █ 
      █████    █       █  █   ██    █        █   ██ ██   █        █  ██ █   ███████     ██  █  ██   
 █  █ ██     █      ██   █  █ ██                    ████████ █ █    ██   █  ██   ██       ██   ██    
  █   █████     ██       █    ██  █  ██ █   █      ██  ██   ██  ███ █      ███   █████   █    █     
    ██   ██   █     ██  █ █ █ ██         █   █  █ █  █ ██  █  █ █    ███ █  ██   ██   █   

**root-me ZIP file**


> consider before hitting run button!
>> take about 36 mins



In [ ]:
import zipfile
from itertools import product
CHARACTERS = 
'1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!@#$%^&*()'

def Brute(zipFile,password):
    # Thử giải nén file zip
    try:
        zipFile.extractall(pwd=password.encode())
        print ('Password:',password)
        return True
    except:
        pass

def main():
    # File zip cần crack
    zipFile = zipfile.ZipFile('ch5.zip')
    length = 3

    # Lặp cho đến khi tìm ra pass
    while True:
        # Tạo list pass
        listPass = product(CHARACTERS, repeat=length)
        for tryPass in listPass:
            password = ''.join(tryPass)
            # Nếu password đúng thì return
            if Brute(zipFile,password):
                return
        # Nếu list pass sai, tăng độ dài kí tự trong list lên 1
        length += 1

main()

Password: 14535


**root-me: Discrete logarithm problem**

In [ ]:
from sympy.ntheory.residue_ntheory import discrete_log

p = 7863166752583943287208453249445887802885958578827520225154826621191353388988908983484279021978114049838254701703424499688950361788140197906625796305008451719

y = 6289736695712027841545587266292164172813699099085672937550442102159309081155467550411414088175729823598108452032137447608687929628597035278365152781494883808

g = 2862392356922936880157505726961027620297475166595443090826668842052108260396755078180089295033677131286733784955854335672518017968622162153227778875458650593

#log_g(y)(mod p)>> discrete_log(modulo, f(x), base)

print(discrete_log(p, y, g))

5984395521967467346233654974364351165422250974773836380349996370549071399526484130546437803585190342858690941215911884205493540794098544612334027224908987632


**RSA - Factorisation**

---




**Android Pattern**

In [ ]:
import hashlib

varchar = [b'\x00', b'\x01', b'\x02', b'\x03', b'\x04', b'\x05',\
           b'\x06', b'\x07', b'\x08']

def make_paths(path):
  if len(path) >= 4:
    key = hashlib.sha1(path).hexdigest()
    if key == '2c3422d33fb9dd9cde87657408e48f4e635713cb': print(path)

  for x in varchar:
    if x not in path:
      make_paths(path + x)

for x in varchar:
  make_paths(x)

b'\x01\x04\x05\x02\x06\x03\x07\x08\x00'


**RSA Decipher Oracle**

In [ ]:
#RSA Decipher Oracle

import binascii

n=456378902858290907415273676326459758501863587455889046415299414290812776158851091008643992243505529957417209835882169153356466939122622249355759661863573516345589069208441886191855002128064647429111920432377907516007825359999
e=65537
c=41662410494900335978865720133929900027297481493143223026704112339997247425350599249812554512606167456298217619549359408254657263874918458518753744624966096201608819511858664268685529336163181156329400702800322067190861310616

#m1=x*m

x=2 
#let x = 2 => m1 = 2m

c1=(c*(pow(x,e)%n))%n
print(f'c1 = {c1}')
m1=1458414996286361336171022694278618963923904557163518886908519150023343172644758356895532151523030

m=m1//2

plaint_text=hex(m)[2:]

print({unhexlify(plaint_text)})



c1 = 403296171084342476768865695356366867028721291703463042604833158549700921098944787236538386192787597217497633027002405570169368914694506837480478155549447010226246552469758067511418934029487541661784304031658335361724392752997
{b'Well done! The flag is S1d3Ch4nn3l4tt4ck'}


**RSA - Factorisation**

In [ ]:
!pip install pycryptodome
#RSA - Factorisation
import base64
import binascii
from binascii import hexlify,unhexlify

def modinv(e, _phi):
  d = -1
  y0, y1, phi = 0, 1, _phi
  while (e > 0):
    r = phi % e
    if r == 0:
      break
    q = phi//e
    d = y0 - y1*q
    phi, e, y0, y1 = e, r, y1, d
  while d < 0:
    d = d + _phi
  return d


In [ ]:
#!pip install factordb-pycli
from factordb.factordb import FactorDB
n = (int('00c2cbb24fdbf923b61268e3f11a3896de4574b3ba58730cbd652938864e2223eeeb704a17cfd08d16b46891a61474759939c6e49aafe7f2595548c74c1d7fb8d24cd15cb23b4cd0a3',16))
f=FactorDB(n)
f.connect()
fctr_list = f.get_factor_list()

p= fctr_list[0]
q = fctr_list[1]


In [ ]:
n = (int('00c2cbb24fdbf923b61268e3f11a3896de4574b3ba58730cbd652938864e2223eeeb704a17cfd08d16b46891a61474759939c6e49aafe7f2595548c74c1d7fb8d24cd15cb23b4cd0a3',16))
e = int('10001', 16)

phi = (p-1)*(q-1)
d = modinv(e,phi)

c = base64.b64decode("e8oQDihsmkvjT3sZe+EE8lwNvBEsFegYF6+OOFOiR6gMtMZxxba/bIgLUD8pV3yEf0gOOfHuB5bC3vQmo7bE4PcIKfpFGZBA")

cipher = int(hexlify(c),16)

plaintext = pow(cipher,d,n)

plaintext = hex(plaintext)[2:]

if len(plaintext)%2:
        plaintext="0"+plaintext

print(plaintext)
print(binascii.a2b_hex(plaintext)[-15:-1])

02159b07401a5dfe9326a7a8dc5471ed8d3f93a65616db0916aecf1332550254fed5416522689e476b02141dee60522186e06065ffd8f0007570326c36446e6149685a6778410a
b'up2l6DnaIhZgxA'
